In [72]:
import pandas as pd
import regex as re
import numpy as nt
import os, csv, nltk, lda
import pandas as pd
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [73]:
data = pd.read_csv('mms.csv')
df = pd.DataFrame(data.loc[data['links'].map(lambda x: len(re.findall("PM's address[^&]+",x))>0),'text'])
df['author'] = 'MMS'

In [74]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
reviews_list = df["text"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
df['text'] = reviews_list

[nltk_data] Downloading package stopwords to /Users/harsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/harsh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/harsh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [75]:
name = input('provide the column name for id: ')
text = input('provide the column name for text: ')
ntopics= input('Provide the number of latent topics: ')

provide the column name for id: author
provide the column name for text: text
Provide the number of latent topics: 10


In [76]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(df[text])

print(total_features_words.shape)

(298, 14284)


In [77]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
df=df.join(doc_topic)
output=pd.DataFrame()



INFO:lda:n_documents: 298
INFO:lda:vocab_size: 14284
INFO:lda:n_words: 219948
INFO:lda:n_topics: 10
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -2450473
INFO:lda:<10> log likelihood: -2002528
INFO:lda:<20> log likelihood: -1915650
INFO:lda:<30> log likelihood: -1892745
INFO:lda:<40> log likelihood: -1881222
INFO:lda:<50> log likelihood: -1874782
INFO:lda:<60> log likelihood: -1870382
INFO:lda:<70> log likelihood: -1865442
INFO:lda:<80> log likelihood: -1863602
INFO:lda:<90> log likelihood: -1863162
INFO:lda:<100> log likelihood: -1860830
INFO:lda:<110> log likelihood: -1858453
INFO:lda:<120> log likelihood: -1858094
INFO:lda:<130> log likelihood: -1855878
INFO:lda:<140> log likelihood: -1854439
INFO:lda:<150> log likelihood: -1853532
INFO:lda:<160> log likelihood: -1852230
INFO:lda:<170> log likelihood: -1852840
INFO:lda:<180> log likelihood: -1851010
INFO:lda:<190> log likelihood: -1850374
INFO:lda:<200> log likelihood: -1850887
INFO:lda:<210> log likelihood: -1849588
INFO:lda:<

In [78]:
for i in range(int(ntopics)):
    topic="topic_"+str(i)
    output[topic]=df.groupby([name])[i].mean()

output=output.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
output.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


In [81]:
"""
If you use the VADER sentiment analysis tools, please cite:
Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
Sentiment Analysis of Social Media Text. Eighth International Conference on
Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
"""
import os
import re
import math
import string
import codecs
import json
from itertools import product
from inspect import getsourcefile
from io import open

# ##Constants##

# (empirically derived mean sentiment intensity rating increase for booster words)
B_INCR = 0.293
B_DECR = -0.293

# (empirically derived mean sentiment intensity rating increase for using ALLCAPs to emphasize a word)
C_INCR = 0.733
N_SCALAR = -0.74

NEGATE = \
    ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

# booster/dampener 'intensifiers' or 'degree adverbs'
# http://en.wiktionary.org/wiki/Category:English_degree_adverbs

BOOSTER_DICT = \
    {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR, 
     "completely": B_INCR, "considerable": B_INCR, "considerably": B_INCR,
     "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormous": B_INCR, "enormously": B_INCR,
     "entirely": B_INCR, "especially": B_INCR, "exceptional": B_INCR, "exceptionally": B_INCR, 
     "extreme": B_INCR, "extremely": B_INCR,
     "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR, "frackin": B_INCR, "fracking": B_INCR,
     "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR, 
     "fuckin": B_INCR, "fucking": B_INCR, "fuggin": B_INCR, "fugging": B_INCR,
     "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR, 
     "incredible": B_INCR, "incredibly": B_INCR, "intensely": B_INCR, 
     "major": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
     "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
     "so": B_INCR, "substantially": B_INCR,
     "thoroughly": B_INCR, "total": B_INCR, "totally": B_INCR, "tremendous": B_INCR, "tremendously": B_INCR,
     "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utter": B_INCR, "utterly": B_INCR,
     "very": B_INCR,
     "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
     "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
     "less": B_DECR, "little": B_DECR, "marginal": B_DECR, "marginally": B_DECR,
     "occasional": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
     "scarce": B_DECR, "scarcely": B_DECR, "slight": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
     "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}

# check for sentiment laden idioms that do not contain lexicon words (future work, not yet implemented)
SENTIMENT_LADEN_IDIOMS = {"cut the mustard": 2, "hand to mouth": -2,
                          "back handed": -2, "blow smoke": -2, "blowing smoke": -2,
                          "upper hand": 1, "break a leg": 2,
                          "cooking with gas": 2, "in the black": 2, "in the red": -2,
                          "on the ball": 2, "under the weather": -2}

# check for special case idioms and phrases containing lexicon words
SPECIAL_CASES = {"the shit": 3, "the bomb": 3, "bad ass": 1.5, "badass": 1.5, "bus stop": 0.0,
                 "yeah right": -2, "kiss of death": -1.5, "to die for": 3, 
                 "beating heart": 3.1, "broken heart": -2.9 }


# #Static methods# #

def negated(input_words, include_nt=True):
    """
    Determine if input contains negation words
    """
    input_words = [str(w).lower() for w in input_words]
    neg_words = []
    neg_words.extend(NEGATE)
    for word in neg_words:
        if word in input_words:
            return True
    if include_nt:
        for word in input_words:
            if "n't" in word:
                return True
    '''if "least" in input_words:
        i = input_words.index("least")
        if i > 0 and input_words[i - 1] != "at":
            return True'''
    return False


def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score / math.sqrt((score * score) + alpha)
    if norm_score < -1.0:
        return -1.0
    elif norm_score > 1.0:
        return 1.0
    else:
        return norm_score


def allcap_differential(words):
    """
    Check whether just some words in the input are ALL CAPS
    :param list words: The words to inspect
    :returns: `True` if some but not all items in `words` are ALL CAPS
    """
    is_different = False
    allcap_words = 0
    for word in words:
        if word.isupper():
            allcap_words += 1
    cap_differential = len(words) - allcap_words
    if 0 < cap_differential < len(words):
        is_different = True
    return is_different


def scalar_inc_dec(word, valence, is_cap_diff):
    """
    Check if the preceding words increase, decrease, or negate/nullify the
    valence
    """
    scalar = 0.0
    word_lower = word.lower()
    if word_lower in BOOSTER_DICT:
        scalar = BOOSTER_DICT[word_lower]
        if valence < 0:
            scalar *= -1
        # check if booster/dampener word is in ALLCAPS (while others aren't)
        if word.isupper() and is_cap_diff:
            if valence > 0:
                scalar += C_INCR
            else:
                scalar -= C_INCR
    return scalar


class SentiText(object):
    """
    Identify sentiment-relevant string-level properties of input text.
    """

    def __init__(self, text):
        if not isinstance(text, str):
            text = str(text).encode('utf-8')
        self.text = text
        self.words_and_emoticons = self._words_and_emoticons()
        # doesn't separate words from\
        # adjacent punctuation (keeps emoticons & contractions)
        self.is_cap_diff = allcap_differential(self.words_and_emoticons)

    @staticmethod
    def _strip_punc_if_word(token):
        """
        Removes all trailing and leading punctuation
        If the resulting string has two or fewer characters,
        then it was likely an emoticon, so return original string
        (ie ":)" stripped would be "", so just return ":)"
        """
        stripped = token.strip(string.punctuation)
        if len(stripped) <= 2:
            return token
        return stripped

    def _words_and_emoticons(self):
        """
        Removes leading and trailing puncutation
        Leaves contractions and most emoticons
            Does not preserve punc-plus-letter emoticons (e.g. :D)
        """
        wes = self.text.split()
        stripped = list(map(self._strip_punc_if_word, wes))
        return stripped

class SentimentIntensityAnalyzer(object):
    """
    Give a sentiment intensity score to sentences.
    """

    def __init__(self, lexicon_file="vader_lexicon.txt", emoji_lexicon="emoji_utf8_lexicon.txt"):
        _this_module_file_path_ = os.path.abspath(getsourcefile(lambda: 0))
        lexicon_full_filepath = os.path.join(os.path.dirname(_this_module_file_path_), lexicon_file)
        with codecs.open(lexicon_full_filepath, encoding='utf-8') as f:
            self.lexicon_full_filepath = f.read()
        self.lexicon = self.make_lex_dict()

        emoji_full_filepath = os.path.join(os.path.dirname(_this_module_file_path_), emoji_lexicon)
        with codecs.open(emoji_full_filepath, encoding='utf-8') as f:
            self.emoji_full_filepath = f.read()
        self.emojis = self.make_emoji_dict()

    def make_lex_dict(self):
        """
        Convert lexicon file to a dictionary
        """
        lex_dict = {}
        for line in self.lexicon_full_filepath.rstrip('\n').split('\n'):
            if not line:
                continue
            (word, measure) = line.strip().split('\t')[0:2]
            lex_dict[word] = float(measure)
        return lex_dict

    def make_emoji_dict(self):
        """
        Convert emoji lexicon file to a dictionary
        """
        emoji_dict = {}
        for line in self.emoji_full_filepath.rstrip('\n').split('\n'):
            (emoji, description) = line.strip().split('\t')[0:2]
            emoji_dict[emoji] = description
        return emoji_dict

    def polarity_scores(self, text):
        """
        Return a float for sentiment strength based on the input text.
        Positive values are positive valence, negative value are negative
        valence.
        """
        # convert emojis to their textual descriptions
        text_no_emoji = ""
        prev_space = True
        for chr in text:
            if chr in self.emojis:
                # get the textual description
                description = self.emojis[chr]
                if not prev_space:
                    text_no_emoji += ' '
                text_no_emoji += description
                prev_space = False
            else:
                text_no_emoji += chr
                prev_space = chr == ' '
        text = text_no_emoji.strip()

        sentitext = SentiText(text)

        sentiments = []
        words_and_emoticons = sentitext.words_and_emoticons
        for i, item in enumerate(words_and_emoticons):
            valence = 0
            # check for vader_lexicon words that may be used as modifiers or negations
            if item.lower() in BOOSTER_DICT:
                sentiments.append(valence)
                continue
            if (i < len(words_and_emoticons) - 1 and item.lower() == "kind" and
                    words_and_emoticons[i + 1].lower() == "of"):
                sentiments.append(valence)
                continue

            sentiments = self.sentiment_valence(valence, sentitext, item, i, sentiments)

        sentiments = self._but_check(words_and_emoticons, sentiments)

        valence_dict = self.score_valence(sentiments, text)

        return valence_dict

    def sentiment_valence(self, valence, sentitext, item, i, sentiments):
        is_cap_diff = sentitext.is_cap_diff
        words_and_emoticons = sentitext.words_and_emoticons
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            # get the sentiment valence 
            valence = self.lexicon[item_lowercase]
                
            # check for "no" as negation for an adjacent lexicon item vs "no" as its own stand-alone lexicon item
            if item_lowercase == "no" and i != len(words_and_emoticons)-1 and words_and_emoticons[i + 1].lower() in self.lexicon:
                # don't use valence of "no" as a lexicon item. Instead set it's valence to 0.0 and negate the next item
                valence = 0.0
            if (i > 0 and words_and_emoticons[i - 1].lower() == "no") \
               or (i > 1 and words_and_emoticons[i - 2].lower() == "no") \
               or (i > 2 and words_and_emoticons[i - 3].lower() == "no" and words_and_emoticons[i - 1].lower() in ["or", "nor"] ):
                valence = self.lexicon[item_lowercase] * N_SCALAR
            
            # check if sentiment laden word is in ALL CAPS (while others aren't)
            if item.isupper() and is_cap_diff:
                if valence > 0:
                    valence += C_INCR
                else:
                    valence -= C_INCR

            for start_i in range(0, 3):
                # dampen the scalar modifier of preceding words and emoticons
                # (excluding the ones that immediately preceed the item) based
                # on their distance from the current item.
                if i > start_i and words_and_emoticons[i - (start_i + 1)].lower() not in self.lexicon:
                    s = scalar_inc_dec(words_and_emoticons[i - (start_i + 1)], valence, is_cap_diff)
                    if start_i == 1 and s != 0:
                        s = s * 0.95
                    if start_i == 2 and s != 0:
                        s = s * 0.9
                    valence = valence + s
                    valence = self._negation_check(valence, words_and_emoticons, start_i, i)
                    if start_i == 2:
                        valence = self._special_idioms_check(valence, words_and_emoticons, i)

            valence = self._least_check(valence, words_and_emoticons, i)
        sentiments.append(valence)
        return sentiments

    def _least_check(self, valence, words_and_emoticons, i):
        # check for negation case using "least"
        if i > 1 and words_and_emoticons[i - 1].lower() not in self.lexicon \
                and words_and_emoticons[i - 1].lower() == "least":
            if words_and_emoticons[i - 2].lower() != "at" and words_and_emoticons[i - 2].lower() != "very":
                valence = valence * N_SCALAR
        elif i > 0 and words_and_emoticons[i - 1].lower() not in self.lexicon \
                and words_and_emoticons[i - 1].lower() == "least":
            valence = valence * N_SCALAR
        return valence

    @staticmethod
    def _but_check(words_and_emoticons, sentiments):
        # check for modification in sentiment due to contrastive conjunction 'but'
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        if 'but' in words_and_emoticons_lower:
            bi = words_and_emoticons_lower.index('but')
            for sentiment in sentiments:
                si = sentiments.index(sentiment)
                if si < bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment * 0.5)
                elif si > bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment * 1.5)
        return sentiments

    @staticmethod
    def _special_idioms_check(valence, words_and_emoticons, i):
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        onezero = "{0} {1}".format(words_and_emoticons_lower[i - 1], words_and_emoticons_lower[i])

        twoonezero = "{0} {1} {2}".format(words_and_emoticons_lower[i - 2],
                                          words_and_emoticons_lower[i - 1], words_and_emoticons_lower[i])

        twoone = "{0} {1}".format(words_and_emoticons_lower[i - 2], words_and_emoticons_lower[i - 1])

        threetwoone = "{0} {1} {2}".format(words_and_emoticons_lower[i - 3],
                                           words_and_emoticons_lower[i - 2], words_and_emoticons_lower[i - 1])

        threetwo = "{0} {1}".format(words_and_emoticons_lower[i - 3], words_and_emoticons_lower[i - 2])

        sequences = [onezero, twoonezero, twoone, threetwoone, threetwo]

        for seq in sequences:
            if seq in SPECIAL_CASES:
                valence = SPECIAL_CASES[seq]
                break

        if len(words_and_emoticons_lower) - 1 > i:
            zeroone = "{0} {1}".format(words_and_emoticons_lower[i], words_and_emoticons_lower[i + 1])
            if zeroone in SPECIAL_CASES:
                valence = SPECIAL_CASES[zeroone]
        if len(words_and_emoticons_lower) - 1 > i + 1:
            zeroonetwo = "{0} {1} {2}".format(words_and_emoticons_lower[i], words_and_emoticons_lower[i + 1],
                                              words_and_emoticons_lower[i + 2])
            if zeroonetwo in SPECIAL_CASES:
                valence = SPECIAL_CASES[zeroonetwo]

        # check for booster/dampener bi-grams such as 'sort of' or 'kind of'
        n_grams = [threetwoone, threetwo, twoone]
        for n_gram in n_grams:
            if n_gram in BOOSTER_DICT:
                valence = valence + BOOSTER_DICT[n_gram]
        return valence

    @staticmethod
    def _sentiment_laden_idioms_check(valence, senti_text_lower):
        # Future Work
        # check for sentiment laden idioms that don't contain a lexicon word
        idioms_valences = []
        for idiom in SENTIMENT_LADEN_IDIOMS:
            if idiom in senti_text_lower:
                print(idiom, senti_text_lower)
                valence = SENTIMENT_LADEN_IDIOMS[idiom]
                idioms_valences.append(valence)
        if len(idioms_valences) > 0:
            valence = sum(idioms_valences) / float(len(idioms_valences))
        return valence

    @staticmethod
    def _negation_check(valence, words_and_emoticons, start_i, i):
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        if start_i == 0:
            if negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 1 word preceding lexicon word (w/o stopwords)
                valence = valence * N_SCALAR
        if start_i == 1:
            if words_and_emoticons_lower[i - 2] == "never" and \
                    (words_and_emoticons_lower[i - 1] == "so" or
                     words_and_emoticons_lower[i - 1] == "this"):
                valence = valence * 1.25
            elif words_and_emoticons_lower[i - 2] == "without" and \
                    words_and_emoticons_lower[i - 1] == "doubt":
                valence = valence
            elif negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 2 words preceding the lexicon word position
                valence = valence * N_SCALAR
        if start_i == 2:
            if words_and_emoticons_lower[i - 3] == "never" and \
                    (words_and_emoticons_lower[i - 2] == "so" or words_and_emoticons_lower[i - 2] == "this") or \
                    (words_and_emoticons_lower[i - 1] == "so" or words_and_emoticons_lower[i - 1] == "this"):
                valence = valence * 1.25
            elif words_and_emoticons_lower[i - 3] == "without" and \
                    (words_and_emoticons_lower[i - 2] == "doubt" or words_and_emoticons_lower[i - 1] == "doubt"):
                valence = valence
            elif negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 3 words preceding the lexicon word position
                valence = valence * N_SCALAR
        return valence

    def _punctuation_emphasis(self, text):
        # add emphasis from exclamation points and question marks
        ep_amplifier = self._amplify_ep(text)
        qm_amplifier = self._amplify_qm(text)
        punct_emph_amplifier = ep_amplifier + qm_amplifier
        return punct_emph_amplifier

    @staticmethod
    def _amplify_ep(text):
        # check for added emphasis resulting from exclamation points (up to 4 of them)
        ep_count = text.count("!")
        if ep_count > 4:
            ep_count = 4
        # (empirically derived mean sentiment intensity rating increase for
        # exclamation points)
        ep_amplifier = ep_count * 0.292
        return ep_amplifier

    @staticmethod
    def _amplify_qm(text):
        # check for added emphasis resulting from question marks (2 or 3+)
        qm_count = text.count("?")
        qm_amplifier = 0
        if qm_count > 1:
            if qm_count <= 3:
                # (empirically derived mean sentiment intensity rating increase for
                # question marks)
                qm_amplifier = qm_count * 0.18
            else:
                qm_amplifier = 0.96
        return qm_amplifier

    @staticmethod
    def _sift_sentiment_scores(sentiments):
        # want separate positive versus negative sentiment scores
        pos_sum = 0.0
        neg_sum = 0.0
        neu_count = 0
        for sentiment_score in sentiments:
            if sentiment_score > 0:
                pos_sum += (float(sentiment_score) + 1)  # compensates for neutral words that are counted as 1
            if sentiment_score < 0:
                neg_sum += (float(sentiment_score) - 1)  # when used with math.fabs(), compensates for neutrals
            if sentiment_score == 0:
                neu_count += 1
        return pos_sum, neg_sum, neu_count

    def score_valence(self, sentiments, text):
        if sentiments:
            sum_s = float(sum(sentiments))
            # compute and add emphasis from punctuation in text
            punct_emph_amplifier = self._punctuation_emphasis(text)
            if sum_s > 0:
                sum_s += punct_emph_amplifier
            elif sum_s < 0:
                sum_s -= punct_emph_amplifier

            compound = normalize(sum_s)
            # discriminate between positive, negative and neutral sentiment scores
            pos_sum, neg_sum, neu_count = self._sift_sentiment_scores(sentiments)

            if pos_sum > math.fabs(neg_sum):
                pos_sum += punct_emph_amplifier
            elif pos_sum < math.fabs(neg_sum):
                neg_sum -= punct_emph_amplifier

            total = pos_sum + math.fabs(neg_sum) + neu_count
            pos = math.fabs(pos_sum / total)
            neg = math.fabs(neg_sum / total)
            neu = math.fabs(neu_count / total)

        else:
            compound = 0.0
            pos = 0.0
            neg = 0.0
            neu = 0.0

        sentiment_dict = \
            {"neg": round(neg, 3),
             "neu": round(neu, 3),
             "pos": round(pos, 3),
             "compound": round(compound, 4)}

        return sentiment_dict




----------------------------------------------------
 - Analyze typical example cases, including handling of:
  -- negations
  -- punctuation emphasis & punctuation flooding
  -- word-shape as emphasis (capitalization difference)
  -- degree modifiers (intensifiers such as 'very' and dampeners such as 'kind of')
  -- slang words as modifiers such as 'uber' or 'friggin' or 'kinda'
  -- contrastive conjunction 'but' indicating a shift in sentiment; sentiment of later text is dominant
  -- use of contractions as negations
  -- sentiment laden emoticons such as :) and :D
  -- utf-8 encoded emojis such as 💘 and 💋 and 😁
  -- sentiment laden slang words (e.g., 'sux')
  -- sentiment laden initialisms and acronyms (for example: 'lol') 

VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!----------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is

In [87]:
speeches = data['text'].values
for i in speeches:
    vs = analyzer.polarity_scores(i)
    print(vs)

{'neg': 0.0, 'neu': 0.722, 'pos': 0.278, 'compound': 0.9986}
{'neg': 0.043, 'neu': 0.793, 'pos': 0.164, 'compound': 0.9998}
{'neg': 0.025, 'neu': 0.817, 'pos': 0.157, 'compound': 0.9998}
{'neg': 0.068, 'neu': 0.732, 'pos': 0.2, 'compound': 0.9995}
{'neg': 0.051, 'neu': 0.75, 'pos': 0.2, 'compound': 0.9999}
{'neg': 0.044, 'neu': 0.781, 'pos': 0.176, 'compound': 0.9998}
{'neg': 0.035, 'neu': 0.868, 'pos': 0.096, 'compound': 0.9452}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.051, 'neu': 0.799, 'pos': 0.15, 'compound': 0.9995}
{'neg': 0.11, 'neu': 0.733, 'pos': 0.156, 'compound': 0.9972}
{'neg': 0.045, 'neu': 0.78, 'pos': 0.174, 'compound': 0.9999}
{'neg': 0.014, 'neu': 0.778, 'pos': 0.208, 'compound': 0.9997}
{'neg': 0.094, 'neu': 0.708, 'pos': 0.198, 'compound': 0.9992}
{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'compound': 0.9987}
{'neg': 0.079, 'neu': 0.724, 'pos': 0.196, 'compound': 0.9987}
{'neg': 0.024, 'neu': 0.786, 'pos': 0.191, 'compound': 0.9998}
{'neg': 0.097

{'neg': 0.022, 'neu': 0.841, 'pos': 0.138, 'compound': 0.9993}
{'neg': 0.044, 'neu': 0.745, 'pos': 0.211, 'compound': 0.9999}
{'neg': 0.027, 'neu': 0.71, 'pos': 0.263, 'compound': 0.9998}
{'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'compound': 0.9941}
{'neg': 0.029, 'neu': 0.746, 'pos': 0.225, 'compound': 0.9984}
{'neg': 0.076, 'neu': 0.774, 'pos': 0.151, 'compound': 0.9954}
{'neg': 0.052, 'neu': 0.818, 'pos': 0.129, 'compound': 0.9994}
{'neg': 0.009, 'neu': 0.804, 'pos': 0.187, 'compound': 0.9998}
{'neg': 0.15, 'neu': 0.661, 'pos': 0.189, 'compound': 0.8518}
{'neg': 0.076, 'neu': 0.758, 'pos': 0.166, 'compound': 0.9994}
{'neg': 0.031, 'neu': 0.82, 'pos': 0.148, 'compound': 0.9996}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.027, 'neu': 0.816, 'pos': 0.157, 'compound': 0.9996}
{'neg': 0.06, 'neu': 0.784, 'pos': 0.155, 'compound': 0.9996}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.012, 'neu': 0.785, 'pos': 0.204, 'compound': 0.9989}
{'neg': 0.02, 'n

{'neg': 0.104, 'neu': 0.743, 'pos': 0.152, 'compound': 0.9981}
{'neg': 0.066, 'neu': 0.766, 'pos': 0.168, 'compound': 0.9998}
{'neg': 0.036, 'neu': 0.797, 'pos': 0.167, 'compound': 0.9997}
{'neg': 0.01, 'neu': 0.883, 'pos': 0.108, 'compound': 0.8989}
{'neg': 0.025, 'neu': 0.789, 'pos': 0.186, 'compound': 0.9983}
{'neg': 0.059, 'neu': 0.8, 'pos': 0.141, 'compound': 0.9992}
{'neg': 0.018, 'neu': 0.788, 'pos': 0.194, 'compound': 0.9995}
{'neg': 0.022, 'neu': 0.829, 'pos': 0.149, 'compound': 0.9982}
{'neg': 0.077, 'neu': 0.79, 'pos': 0.133, 'compound': 0.9994}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.124, 'neu': 0.723, 'pos': 0.153, 'compound': 0.97}
{'neg': 0.033, 'neu': 0.802, 'pos': 0.165, 'compound': 0.9983}
{'neg': 0.043, 'neu': 0.801, 'pos': 0.156, 'compound': 0.9993}
{'neg': 0.011, 'neu': 0.803, 'pos': 0.185, 'compound': 0.9996}
{'neg': 0.006, 'neu': 0.799, 'pos': 0.196, 'compound': 0.9985}
{'neg': 0.043, 'neu': 0.711, 'pos': 0.246, 'compound': 0.9995}
{'neg':

{'neg': 0.024, 'neu': 0.783, 'pos': 0.193, 'compound': 0.9995}
{'neg': 0.175, 'neu': 0.678, 'pos': 0.146, 'compound': -0.9969}
{'neg': 0.054, 'neu': 0.756, 'pos': 0.19, 'compound': 0.9999}
{'neg': 0.075, 'neu': 0.756, 'pos': 0.17, 'compound': 0.9998}
{'neg': 0.041, 'neu': 0.768, 'pos': 0.191, 'compound': 0.9963}
{'neg': 0.047, 'neu': 0.861, 'pos': 0.092, 'compound': 0.6629}
{'neg': 0.006, 'neu': 0.778, 'pos': 0.217, 'compound': 0.9992}
{'neg': 0.034, 'neu': 0.76, 'pos': 0.206, 'compound': 0.9999}
{'neg': 0.05, 'neu': 0.791, 'pos': 0.159, 'compound': 0.9997}
{'neg': 0.107, 'neu': 0.805, 'pos': 0.088, 'compound': -0.7089}
{'neg': 0.055, 'neu': 0.761, 'pos': 0.184, 'compound': 0.9999}
{'neg': 0.036, 'neu': 0.773, 'pos': 0.191, 'compound': 0.9999}
{'neg': 0.028, 'neu': 0.776, 'pos': 0.196, 'compound': 0.9999}
{'neg': 0.006, 'neu': 0.817, 'pos': 0.177, 'compound': 0.999}
{'neg': 0.053, 'neu': 0.773, 'pos': 0.174, 'compound': 0.9912}
{'neg': 0.051, 'neu': 0.813, 'pos': 0.136, 'compound': 0.9

{'neg': 0.015, 'neu': 0.772, 'pos': 0.212, 'compound': 0.9998}
{'neg': 0.062, 'neu': 0.802, 'pos': 0.136, 'compound': 0.9942}
{'neg': 0.028, 'neu': 0.811, 'pos': 0.161, 'compound': 0.9996}
{'neg': 0.012, 'neu': 0.829, 'pos': 0.158, 'compound': 0.9999}
{'neg': 0.0, 'neu': 0.72, 'pos': 0.28, 'compound': 0.5423}
{'neg': 0.048, 'neu': 0.749, 'pos': 0.203, 'compound': 0.9985}
{'neg': 0.042, 'neu': 0.883, 'pos': 0.075, 'compound': 0.9918}
{'neg': 0.022, 'neu': 0.753, 'pos': 0.224, 'compound': 0.9979}
{'neg': 0.035, 'neu': 0.754, 'pos': 0.211, 'compound': 0.9995}
{'neg': 0.046, 'neu': 0.786, 'pos': 0.168, 'compound': 0.9997}
{'neg': 0.038, 'neu': 0.788, 'pos': 0.174, 'compound': 0.9994}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.012, 'neu': 0.845, 'pos': 0.143, 'compound': 0.9994}
{'neg': 0.007, 'neu': 0.993, 'pos': 0.0, 'compound': -0.8689}
{'neg': 0.0, 'neu': 0.999, 'pos': 0.001, 'compound': 0.0772}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.021, 'n

...

{'neg': 0.044, 'neu': 0.778, 'pos': 0.178, 'compound': 1.0}
{'neg': 0.026, 'neu': 0.841, 'pos': 0.133, 'compound': 0.9995}
{'neg': 0.031, 'neu': 0.757, 'pos': 0.212, 'compound': 0.9973}
{'neg': 0.052, 'neu': 0.813, 'pos': 0.134, 'compound': 0.9916}
{'neg': 0.139, 'neu': 0.737, 'pos': 0.124, 'compound': -0.4927}
{'neg': 0.015, 'neu': 0.806, 'pos': 0.178, 'compound': 0.9998}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.022, 'neu': 0.821, 'pos': 0.158, 'compound': 0.9931}
{'neg': 0.018, 'neu': 0.805, 'pos': 0.178, 'compound': 0.9992}
{'neg': 0.002, 'neu': 0.846, 'pos': 0.152, 'compound': 0.9984}
{'neg': 0.014, 'neu': 0.803, 'pos': 0.183, 'compound': 0.9976}
{'neg': 0.035, 'neu': 0.882, 'pos': 0.083, 'compound': 0.996}
{'neg': 0.016, 'neu': 0.767, 'pos': 0.218, 'compound': 0.9999}
{'neg': 0.011, 'neu': 0.809, 'pos': 0.179, 'compound': 0.9997}
{'neg': 0.028, 'neu': 0.841, 'pos': 0.131, 'compound': 0.9996}
{'neg': 0.009, 'neu': 0.689, 'pos': 0.302, 'compound': 0.9997}
{'ne

{'neg': 0.009, 'neu': 0.812, 'pos': 0.179, 'compound': 0.9953}
{'neg': 0.015, 'neu': 0.86, 'pos': 0.125, 'compound': 0.9979}
{'neg': 0.022, 'neu': 0.787, 'pos': 0.191, 'compound': 0.9962}
{'neg': 0.013, 'neu': 0.807, 'pos': 0.179, 'compound': 0.998}
{'neg': 0.011, 'neu': 0.848, 'pos': 0.141, 'compound': 0.9992}
{'neg': 0.012, 'neu': 0.792, 'pos': 0.196, 'compound': 0.999}
{'neg': 0.036, 'neu': 0.73, 'pos': 0.234, 'compound': 0.9999}
{'neg': 0.009, 'neu': 0.824, 'pos': 0.167, 'compound': 0.9992}
{'neg': 0.018, 'neu': 0.889, 'pos': 0.093, 'compound': 0.9704}
{'neg': 0.048, 'neu': 0.853, 'pos': 0.099, 'compound': 0.9961}
{'neg': 0.026, 'neu': 0.778, 'pos': 0.196, 'compound': 0.9986}
{'neg': 0.062, 'neu': 0.812, 'pos': 0.126, 'compound': 0.9986}
{'neg': 0.014, 'neu': 0.769, 'pos': 0.217, 'compound': 0.999}
{'neg': 0.034, 'neu': 0.725, 'pos': 0.241, 'compound': 0.9998}
{'neg': 0.034, 'neu': 0.804, 'pos': 0.162, 'compound': 0.9998}
{'neg': 0.027, 'neu': 0.867, 'pos': 0.106, 'compound': 0.97}

{'neg': 0.022, 'neu': 0.79, 'pos': 0.188, 'compound': 0.9995}
{'neg': 0.016, 'neu': 0.775, 'pos': 0.209, 'compound': 0.9998}
{'neg': 0.024, 'neu': 0.851, 'pos': 0.126, 'compound': 0.9971}
{'neg': 0.001, 'neu': 0.811, 'pos': 0.188, 'compound': 0.9998}
{'neg': 0.028, 'neu': 0.78, 'pos': 0.192, 'compound': 0.9999}
{'neg': 0.021, 'neu': 0.801, 'pos': 0.177, 'compound': 0.9998}
{'neg': 0.042, 'neu': 0.791, 'pos': 0.167, 'compound': 0.9993}
{'neg': 0.032, 'neu': 0.807, 'pos': 0.16, 'compound': 0.9957}
{'neg': 0.083, 'neu': 0.774, 'pos': 0.143, 'compound': 0.9968}
{'neg': 0.011, 'neu': 0.842, 'pos': 0.147, 'compound': 0.9937}
{'neg': 0.056, 'neu': 0.682, 'pos': 0.263, 'compound': 0.9996}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.05, 'neu': 0.752, 'pos': 0.198, 'compound': 0.9999}
{'neg': 0.034, 'neu': 0.771, 'pos': 0.194, 'compound': 0.9999}
{'neg': 0.005, 'neu': 0.826, 'pos': 0.169, 'compound': 0.9984}
{'neg': 0.032, 'neu': 0.817, 'pos': 0.151, 'compound': 0.9998}
{'neg

{'neg': 0.102, 'neu': 0.761, 'pos': 0.137, 'compound': 0.9992}
{'neg': 0.087, 'neu': 0.779, 'pos': 0.134, 'compound': 0.9976}
{'neg': 0.054, 'neu': 0.762, 'pos': 0.184, 'compound': 0.9938}
{'neg': 0.038, 'neu': 0.722, 'pos': 0.24, 'compound': 0.9997}
{'neg': 0.049, 'neu': 0.752, 'pos': 0.199, 'compound': 0.9999}
{'neg': 0.02, 'neu': 0.791, 'pos': 0.19, 'compound': 0.9994}
{'neg': 0.007, 'neu': 0.798, 'pos': 0.195, 'compound': 0.9977}
{'neg': 0.019, 'neu': 0.784, 'pos': 0.197, 'compound': 0.9998}
{'neg': 0.0, 'neu': 0.914, 'pos': 0.086, 'compound': 0.5106}
{'neg': 0.013, 'neu': 0.708, 'pos': 0.279, 'compound': 0.9964}
{'neg': 0.03, 'neu': 0.785, 'pos': 0.185, 'compound': 0.9997}
{'neg': 0.033, 'neu': 0.827, 'pos': 0.14, 'compound': 0.9987}
{'neg': 0.007, 'neu': 0.775, 'pos': 0.217, 'compound': 0.9998}
{'neg': 0.008, 'neu': 0.803, 'pos': 0.19, 'compound': 0.9998}
{'neg': 0.018, 'neu': 0.721, 'pos': 0.261, 'compound': 0.9995}
{'neg': 0.087, 'neu': 0.773, 'pos': 0.14, 'compound': 0.9965}
{

{'neg': 0.015, 'neu': 0.798, 'pos': 0.187, 'compound': 0.9998}
{'neg': 0.039, 'neu': 0.785, 'pos': 0.176, 'compound': 0.9999}
{'neg': 0.057, 'neu': 0.803, 'pos': 0.139, 'compound': 0.9999}
{'neg': 0.027, 'neu': 0.815, 'pos': 0.159, 'compound': 0.9991}
{'neg': 0.025, 'neu': 0.833, 'pos': 0.143, 'compound': 0.9998}
{'neg': 0.05, 'neu': 0.677, 'pos': 0.273, 'compound': 0.9997}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.01, 'neu': 0.793, 'pos': 0.197, 'compound': 0.9989}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.016, 'neu': 0.721, 'pos': 0.263, 'compound': 0.9998}
{'neg': 0.008, 'neu': 0.783, 'pos': 0.21, 'compound': 0.9999}
{'neg': 0.056, 'neu': 0.698, 'pos': 0.246, 'compound': 0.9993}
{'neg': 0.055, 'neu': 0.731, 'pos': 0.214, 'compound': 0.9993}
{'neg': 0.011, 'neu': 0.784, 'pos': 0.205, 'compound': 0.9996}
{'neg': 0.043, 'neu': 0.777, 'pos': 0.18, 'compound': 0.9997}
{'neg': 0.043, 'neu': 0.782, 'pos': 0.175, 'compound': 0.9996}
{'neg': 0.041,

In [84]:
speeches

array(["PM's speech at the launch of the India Japan Friendship Year\n\t\t\t\tHindi VersionI am delighted to welcome you on this happy occasion of the joint launch of the India-Japan Friendship Year 2007, marking the fiftieth anniversary of the Cultural Agreement between India and Japan. 2.\tMore than a thousand years ago, the peoples of India and Japan initiated a vibrant civilizational relationship, when Budhism traveled to Japan. The gentle message that was brought from India to Japan was a message of love, compassion and universal brotherhood. 3.\tThe Festival of India which we launch today will bring out the vibrancy of India's culture in all its diversity; the youthful dynamism of contemporary India's free and open society, and the great transformations that are taking place in India's economy.  4.\tWe are also looking forward to the Festival of Japan in India, which will showcase both modern and traditional Japanese culture, which is catching the popular imagination in India.  5